<a href="https://colab.research.google.com/github/stsushimaHTB/test/blob/master/DataReadFromBigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [0]:
from google.colab import auth
auth.authenticate_user()

EDPS 読み込み

In [0]:
%%bigquery --project htb-10165-01 df_edps_202003
#standardSQL
SELECT
  b030_program_frame_id AS Frame_ID,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', b032_program_start_time, 'Asia/Tokyo') AS Program_Start_jst,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', b033_program_end_time, 'Asia/Tokyo') AS Program_End_jst,
  b036_program_name AS Program_Name,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', b088_epg_start_time, 'Asia/Tokyo') AS EPG_Start_jst,
  b131_station_name AS Station_Name,
  e032_video_source AS Video_Source,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', e026_broadcast_start_time, 'Asia/Tokyo') AS Source_Start_jst,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', e038x_broadcast_end_time, 'Asia/Tokyo') AS Source_End_jst,
  e038_video_time_length AS Length,
  e069_frame_time AS CM_Frame_Length,
  e078_remarks AS remarks,
  e080_cm_flag AS CM_TK_flag,
  e082_sponsor_name AS Sponsor,
  e083_video_name AS Source_Name,
  e086_video_code_10 AS Code10,
  e086x_video_code_8 AS Code08,
  e018_break_flag AS Master_Cut
FROM (
  SELECT
    *
  FROM
    `accesslog_htb.edps_programs_202003*` -- ワイルドカードで4月のテーブルを指定
  WHERE
    b003_service_kind = "SV1"
  ORDER BY
    b032_program_start_time,
    b003_service_kind,
    b030_program_frame_id,
    e026_broadcast_start_time )
WHERE
  e054_special_1 NOT LIKE ("NDAT")
  OR e054_special_1 IS NULL

In [14]:
df_edps_202003.head()

,Frame_ID,Program_Start_jst,Program_End_jst,Program_Name,EPG_Start_jst,Station_Name,Video_Source,Source_Start_jst,Source_End_jst,Length,CM_Frame_Length,remarks,CM_TK_flag,Sponsor,Source_Name,Code10,Code08,Master_Cut
0,0419T,2020-03-01 04:19:00,2020-03-01 04:20:00,オープニング,2020-03-01 04:19:00,HTB,VAF4,2020-03-01 04:19:00,2020-03-01 04:20:00,00:01:00,None,オープニング（新社屋,None,None,オープニング（新社屋移転後）,None,None,None
1,0419T,2020-03-01 04:19:00,2020-03-01 04:20:00,オープニング,2020-03-01 04:19:00,HTB,ED,2020-03-01 04:20:00,2020-03-01 04:20:00,00:00:00,None,None,None,None,None,None,None,None
2,0420T,2020-03-01 04:20:00,2020-03-01 04:49:00,サントリーテレビショッピング,2020-03-01 04:20:00,HTB,PG1,2020-03-01 04:20:00,2020-03-01 04:49:00,00:29:00,None,サントリーテレビショ,None,None,サントリーテレビショッピング,None,None,None
3,0420T,2020-03-01 04:20:00,2020-03-01 04:49:00,サントリーテレビショッピング,2020-03-01 04:20:00,HTB,ED,2020-03-01 04:49:00,2020-03-01 04:49:00,00:00:00,None,None,None,None,None,None,None,None
4,0450S,2020-03-01 04:49:00,2020-03-01 04:50:00,０４：５０ＳＢ,None,None,CM1,2020-03-01 04:49:00,2020-03-01 04:50:00,00:01:00,00:01:00,None,CM,None,None,None,None,None


In [5]:
df_edps_202002

,Frame_ID,Program_Start_jst,Program_End_jst,Program_Name,EPG_Start_jst,Station_Name,Video_Source,Source_Start_jst,Source_End_jst,Length,CM_Frame_Length,remarks,CM_TK_flag,Sponsor,Source_Name,Code10,Code08,Master_Cut
0,0550T,2020-01-01 05:50:00,2020-01-01 05:59:00,ＡＮＮニュース,2020-01-01 05:50:00,EX,H1,2020-01-01 05:50:00,2020-01-01 05:50:05,00:00:05,None,None,None,None,None,None,None,None
1,0550T,2020-01-01 05:50:00,2020-01-01 05:59:00,ＡＮＮニュース,2020-01-01 05:50:00,EX,H1,2020-01-01 05:50:05,2020-01-01 05:50:10,00:00:05,None,None,None,None,None,None,None,None
2,0550T,2020-01-01 05:50:00,2020-01-01 05:59:00,ＡＮＮニュース,2020-01-01 05:50:00,EX,H1,2020-01-01 05:50:10,2020-01-01 05:50:10,00:00:00,None,None,None,None,None,None,None,None
3,0550T,2020-01-01 05:50:00,2020-01-01 05:59:00,ＡＮＮニュース,2020-01-01 05:50:00,EX,CM1,2020-01-01 05:54:15,2020-01-01 05:54:30,00:00:15,None,None,CM,ユーキャン,チャレンジＣＭ 木村佳乃 勉強篇,07GZ20CH01,A49440JV,None
4,0550T,2020-01-01 05:50:00,2020-01-01 05:59:00,ＡＮＮニュース,2020-01-01 05:50:00,EX,CM1,2020-01-01 05:54:15,2020-01-01 05:55:15,00:01:00,00:01:00,None,CM,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37679,2825T,2020-02-01 04:25:00,2020-02-01 04:44:00,朝までＮ天,2020-02-01 04:25:00,HTB,CM1,2020-02-01 04:43:00,2020-02-01 04:43:15,00:00:15,None,None,CM,ＳＵＳ ＆ ＰＲ,【事】１ 博士ちゃんＳＰ １５”,None,None,None
37680,2825T,2020-02-01 04:25:00,2020-02-01 04:44:00,朝までＮ天,2020-02-01 04:25:00,HTB,CM1,2020-02-01 04:43:15,2020-02-01 04:43:45,00:00:30,None,None,CM,ＳＵＳ ＆ ＰＲ,【事】１ アリバイ崩し承り【１／,None,None,None
37681,2825T,2020-02-01 04:25:00,2020-02-01 04:44:00,朝までＮ天,2020-02-01 04:25:00,HTB,CM1,2020-02-01 04:43:45,2020-02-01 04:44:00,00:00:15,None,None,CM,ＳＵＳ ＆ ＰＲ,【レ】ＹｏｕＴｕｂｅチャンネル登,None,None,None
37682,2825T,2020-02-01 04:25:00,2020-02-01 04:44:00,朝までＮ天,2020-02-01 04:25:00,HTB,ED,2020-02-01 04:44:00,2020-02-01 04:44:00,00:00:00,None,None,None,None,None,None,None,None


In [0]:
df_EDPS_hw_202003 = df_edps_202003.query('Program_Name.str.contains("水曜どうでしょう")',engine="python")
df_EDPS_hw_202003.to_csv("/content/drive/My Drive/Colab Notebooks/EDPS_hw_202003.csv")

視聴データ読み込み

1/22から視聴データ取得開始～3/11最終話（データ取得はそれから１週間後くらいまで）


In [0]:
%%bigquery --project htb-10165-01 df_view_202002to202003
#standardSQL
SELECT
  unique_id,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', time_in, 'Asia/Tokyo') AS time_in_jst,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', time_out, 'Asia/Tokyo') AS time_out_jst,
  post_code,
  status_data ,
  ip_address,
  browser,
  FORMAT_TIMESTAMP( '%Y-%m-%d %H:%M:%S', time_send, 'Asia/Tokyo' ) AS time_send_jst,
  manufacture_id ,
  original_data
FROM
  `accesslog_htb.access_log_htb_table`
WHERE
  ( _PARTITIONTIME BETWEEN TIMESTAMP('2020-02-19 04:00:00')
    AND TIMESTAMP('2020-03-19 00:00:00')
    OR _PARTITIONTIME IS NULL)
  AND time_in >= TIMESTAMP('2020-02-19 04:00:00', 'Asia/Tokyo')
  AND time_in < TIMESTAMP('2020-03-19 00:00:00', 'Asia/Tokyo')
ORDER BY tv_id, time_in_jst ASC

In [9]:
df_view_202004.head()

,unique_id,time_in_jst,time_out_jst,post_code,status_data,ip_address,browser,time_send_jst,manufacture_id,original_data
0,a78fcb25-7636-4c4c-bf14-b2ff45f90f67,2020-04-01 13:44:01,2020-04-01 15:26:56,0880560,1,36.3.166.244,KNOSSOSDTG,2020-05-11 16:08:32,12,|2020-04-01T13:44:01|2020-04-01T15:26:56|61|0|...
1,f30380c5-59d3-4293-94cc-b2bd55a972b0,2020-04-03 07:27:34,2020-04-03 07:27:40,0930042,1,111.239.155.176,KNOSSOSDTG,2020-05-11 15:58:40,12,|2020-04-03T07:27:34|2020-04-03T07:27:40|61|0|...
2,e2276df5-6153-4cc0-b36f-6c008b44def6,2020-04-04 08:08:59,2020-04-04 10:06:47,0620938,1,60.59.132.80,iBMLforJC,2020-05-11 16:16:21,24,|2020-04-04T08:08:59|2020-04-04T10:06:47|61|16...
3,3d4fd81d-b7e7-4b77-b19b-e3404592e564,2020-04-04 10:28:39,2020-04-04 10:33:36,0070839,1,60.59.132.211,PANAIBROWSER,2020-05-11 16:00:28,19,|2020-04-04T10:28:39|2020-04-04T10:33:36|61|0|...
4,45d06d12-72bc-42ff-bfbf-27ea7cd304ff,2020-04-04 13:26:04,2020-04-04 13:26:37,0030021,1,126.242.128.221,HD6,2020-05-11 16:01:50,11,|2020-04-04T13:26:04|2020-04-04T13:26:37|61|27...


In [0]:
df_view_202002to202003.to_csv("/content/drive/My Drive/Colab Notebooks/viewlog_hw_202002to202003.csv")

In [0]:
df_view_202004.to_pickle('view_202004.pkl.zip') # kimura 推奨

GCSにアップロード

In [11]:
!gsutil cp /content/view_202004.pkl.zip  gs://access_log_back/from_colab/view_202004.pkl.zip

AccessDeniedException: 403 stsushima@htb.co.jp does not have storage.objects.list access to the Google Cloud Storage bucket.
